In [1]:
import pandas as pd
#import torch
#from transformers import AutoTokenizer, AutoModel
#import re
#import string
import numpy as np
#from nltk.corpus import stopwords 
#from nltk.stem import WordNetLemmatizer
#from nltk.stem import LancasterStemmer
#from simpletransformers.classification import MultiLabelClassificationModel
#import logging
#import custom_sentence_tokenizer
import matplotlib.pyplot as plt
#from scipy import stats
#from ast import literal_eval
import pickle
from keras.preprocessing.sequence import pad_sequences
import keras.backend as K
import keras

Using TensorFlow backend.


In [2]:
#from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM, Reshape
from keras.layers import GlobalMaxPooling1D
from keras.models import Model
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.layers import Input
from keras.layers.merge import Concatenate
#from sklearn.decomposition import PCA
import _pickle as cPickle
import io
import time
import boto3
from boto3.s3.transfer import TransferConfig

In [3]:
def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [4]:
def LoadFromS3(i):
    # https://w210-mimic.s3.amazonaws.com/embeddings1
    now = time.time()
    print("Starting Load for File",i)
    embeddings = {}
    session = boto3.session.Session(region_name='us-east-1')
    s3client = session.client('s3')
    response = s3client.get_object(Bucket='w210-mimic', Key=i)
    body_string = response['Body'].read()
    embeddings.update(cPickle.loads(body_string))
    with open(i, 'wb') as handle:
        pickle.dump(embeddings, handle, protocol=pickle.HIGHEST_PROTOCOL)
    print("File Loaded in:", time.time()-now)
    
    return embeddings

In [5]:
def SaveToS3(file, name):
    # https://w210-mimic.s3.amazonaws.com/embeddings1
    now = time.time()
    print("Starting Write for File", name)
    file = file.to_json()
    session = boto3.session.Session(region_name='us-east-1')
    s3 = boto3.resource('s3')
    object = s3.Object('w210-mimic', name)
    object.put(Body=file)

    print("File Saved in:", time.time()-now)
    
    return

In [6]:
deep_inputs = Input(shape=(3840,768))
LSTM_Layer_1 = LSTM(128)(deep_inputs)
dense_layer_1 = Dense(16, activation='sigmoid')(LSTM_Layer_1)
opt = keras.optimizers.Adam(lr=0.0001)
model = Model(inputs=deep_inputs, outputs=dense_layer_1)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=[get_f1])






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [7]:
EPOCHS = 5

In [ ]:
for epoch in range(EPOCHS):
    for i in ['embeddings1', 'embeddings2', 'embeddings3', 'embeddings4', 'embeddings5', 'embeddings6']:
        data = LoadFromS3(i)
        
        data = np.array(list(data.items()))
        HADM_ID_LIST = data[:,0]
        data = data[:,1]

        data = [np.float16(np.concatenate(i[:30])) for i in data]

        embeddings_padded = []
        
        print("Padding Embeddings:")
        for j in data:
            pad_len = 3840 - len(j)
            if pad_len:
                embeddings_padded.append(np.append(j, np.zeros(pad_len*768, dtype=np.float16).reshape(pad_len, 768), axis=0))
            else:
                embeddings_padded.append(j)
        
        del(data)
        embeddings_padded = np.float16(embeddings_padded)
        embeddings_padded = np.array(embeddings_padded, dtype=np.float16)

        CHAPTER_PIVOT_DF = pd.read_csv('s3://w210-mimic/data/chapter_label.csv', sep=',', header = 0)\
                        .astype({'HADM_ID': 'str'}).set_index('HADM_ID')

        EMBEDDINGS_CHAPTER_PIVOT_DF = pd.DataFrame(data=list(zip(HADM_ID_LIST, embeddings_padded)), columns=['HADM_ID', 'embeddings']).set_index('HADM_ID')\
                    .join(CHAPTER_PIVOT_DF, how='inner')
        EMBEDDINGS_CHAPTER_PIVOT_DF = EMBEDDINGS_CHAPTER_PIVOT_DF.iloc[:,1:]
        EMBEDDINGS_CHAPTER_PIVOT_NP = np.array(EMBEDDINGS_CHAPTER_PIVOT_DF)
        del(EMBEDDINGS_CHAPTER_PIVOT_DF)
        
        print("Splitting Data")

        X_train, X_test, y_train, y_test = train_test_split(embeddings_padded, EMBEDDINGS_CHAPTER_PIVOT_NP, test_size=0.15, random_state=42)#, shuffle=False)
        del(embeddings_padded)
        
        print("Training Epoch:", epoch, "Embeddings:",i)
        
        history = model.fit(X_train, y_train, epochs=1, batch_size=256, verbose=1, validation_split=0.1)
        SaveToS3(model, "brent_chapter_"+epoch+"_"+i)

Starting Load for File embeddings1
File Loaded in: 2803.621426820755
Padding Embeddings:
Splitting Data
Training Epoch: 0 Embeddings: [[ 0.087    0.0953  -0.1653  ... -0.4712  -0.3162  -0.1573 ]
 [-0.1334  -0.032    0.2285  ...  0.1691  -0.05176  0.1978 ]
 [-0.03687 -0.0974   0.1724  ...  0.259   -0.2769  -0.0988 ]
 ...
 [-0.2297  -0.2449  -0.2145  ...  0.1322  -0.2338   0.01703]
 [-0.2046  -0.3538  -0.11475 ...  0.03458 -0.2185   0.1255 ]
 [ 0.3613  -1.016   -0.2155  ... -0.799    1.255   -0.2764 ]]



Train on 7650 samples, validate on 850 samples
Epoch 1/1





1280/7650 [====>.........................] - ETA: 12:06 - loss: 0.6910 - get_f1: 0.4193

# Cells Below are for Debugging & Legacy

In [23]:
EMBEDDINGS_CHAPTER_PIVOT_DF = pd.DataFrame(data=list(zip(HADM_ID_LIST, embeddings_padded)), columns=['HADM_ID', 'embeddings']).set_index('HADM_ID')\
            .join(CHAPTER_PIVOT_DF, how='inner')
EMBEDDINGS_CHAPTER_PIVOT_DF = EMBEDDINGS_CHAPTER_PIVOT_DF.iloc[:,1:]
EMBEDDINGS_CHAPTER_PIVOT_NP = np.array(EMBEDDINGS_CHAPTER_PIVOT_DF)

In [24]:
X_train, X_test, y_train, y_test = train_test_split(embeddings_padded, EMBEDDINGS_CHAPTER_PIVOT_NP, test_size=0.15, random_state=42)

In [ ]:
history = model.fit(X_train, y_train, epochs=1, verbose=1, batch_size=512, validation_split=0.1)

Train on 7650 samples, validate on 850 samples
Epoch 1/1
 512/7650 [=>............................] - ETA: 13:00 - loss: 0.6954 - get_f1: 0.5180

## 1. Import Embeddings (X) and Labels (Y)

### 1.1 Embeddings

In [ ]:
# now = time.time()
# infile = open("embeddings5",'rb')
# embeddings = pickle.load(infile)
# infile.close()
# print(time.time()-now)

In [ ]:
# https://w210-mimic.s3.amazonaws.com/embeddings1
now = time.time()
embeddings = {}
session = boto3.session.Session(region_name='us-east-1')
s3client = session.client('s3')
for i in ['embeddings1']:
          #embeddings3
    response = s3client.get_object(Bucket='w210-mimic', Key=i)
    body_string = response['Body'].read()
    embeddings.update(cPickle.loads(body_string))
    with open(i, 'wb') as handle:
        pickle.dump(embeddings, handle, protocol=pickle.HIGHEST_PROTOCOL)
print(time.time()-now)

In [ ]:
embeddings_np = np.array(list(embeddings.items()))
embeddings_list = None

In [ ]:
embeddings_np[:,1].shape

In [ ]:
embeddings_np.shape

In [ ]:
HADM_ID_LIST = embeddings_np[:,0]
embeddings = embeddings_np[:,1]
embeddings_np = None

In [ ]:
len(embeddings), embeddings[0].shape

In [ ]:
# arr = np.array([embeddings[i].shape[0] for i in range(0,len(embeddings))])
# for i in range(5, 80, 5):
#     print(str(i)+': '+str(len(arr[arr<i])/len(embeddings)))

#### 1.1.1 Zero pad and truncate

In [ ]:
# who

In [ ]:
embeddings_truncated = None
embeddings_truncated = [np.float16(np.concatenate(i[:30])) for i in embeddings]

In [ ]:
30*128

In [ ]:
del(embeddings)
# del(embeddings_truncated)
# del(embeddings_np)
del(embeddings_list)

In [ ]:
embeddings_padded = []
for i in embeddings_truncated:
    pad_len = 3840 - len(i)
    if pad_len:
        embeddings_padded.append(np.append(i, np.zeros(pad_len*768).reshape(pad_len, 768), axis=0))
    else:
        embeddings_padded.append(i)
embeddings_padded = np.array(embeddings_padded)

In [ ]:
embeddings_padded.shape

In [ ]:
del(embeddings_truncated)
del(embeddings_np)

In [ ]:
# who

### 1.2 Labels - Top 100 Dx code and 18 category

In [ ]:
CHAPTER_PIVOT_DF = pd.read_csv('s3://w210-mimic/data/chapter_label.csv', sep=',', header = 0)\
                    .astype({'HADM_ID': 'str'}).set_index('HADM_ID')

In [ ]:
CHAPTER_PIVOT_DF.head()

In [ ]:
HADM_ID_DF = None
HADM_ID_DF =  pd.DataFrame(data=HADM_ID_LIST, columns=['HADM_ID'])

In [ ]:
CHAPTER_PIVOT_DF = HADM_ID_DF.set_index('HADM_ID')\
            .join(CHAPTER_PIVOT_DF, how='left')

In [ ]:
HADM_ID_LIST[:5]

In [ ]:
CHAPTER_PIVOT_NP = np.array(CHAPTER_PIVOT_DF)

In [ ]:
CHAPTER_PIVOT_NP.shape

## 2. Split test and training

### 2.3 Both top-100 ICD9 and category as labels

In [ ]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(embeddings_padded, CHAPTER_PIVOT_NP, test_size=0.15, random_state=42)

In [ ]:
X_train2.shape, X_test2.shape, y_train2.shape, y_test2.shape

## 3. Define LSTM pipeline

### 3.1 LSTM 1

In [ ]:
def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [ ]:
deep_inputs = Input(shape=(3840,768))
LSTM_Layer_1 = LSTM(128)(deep_inputs)
dense_layer_1 = Dense(16, activation='sigmoid')(LSTM_Layer_1)
model = Model(inputs=deep_inputs, outputs=dense_layer_1)

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[get_f1])

In [ ]:
print(model.summary())

In [ ]:
history1 = model.fit(X_train2, y_train2, epochs=3, verbose=1, validation_split=0.1)

In [ ]:
score = model.evaluate(X_test2, y_test2, verbose=1)

print("Test Score:", score[0])
print("Test Accuracy:", score[1])

In [ ]:
pd.DataFrame(y_test2[:,:18]).to_csv('ActualLabels.csv', 
                                   index=False,header=True)
pd.DataFrame(model.predict(X_test2)).to_csv('PredictedLabels.csv', 
                                   index=False,header=True)

### 3.2 LSTM 2

In [ ]:
deep_inputs = Input(shape=(2048,768))
LSTM_Layer_1 = LSTM(128)(deep_inputs)
dense_layer_1 = Dense(64, activation='sigmoid')(LSTM_Layer_1)
dense_layer_2 = Dense(18, activation='sigmoid')(dense_layer_1)
model2 = Model(inputs=deep_inputs, outputs=dense_layer_2)

In [ ]:
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=[get_f1])
history = model2.fit(X_train2, y_train2[:,:18], epochs=5, verbose=1, validation_split=0.1)

In [ ]:
score = model2.evaluate(X_test2, y_test2[:,:18], verbose=1)

print("Test Score:", score[0])
print("Test Accuracy:", score[1])

### 3.3 LSTM 3 with 100 ICD labels

In [ ]:
deep_inputs = Input(shape=(1280,768))
LSTM_Layer_1 = LSTM(128)(deep_inputs)
dense_layer_1 = Dense(256, activation='relu')(LSTM_Layer_1)
dense_layer_2 = Dense(100, activation='sigmoid')(dense_layer_1)
model3 = Model(inputs=deep_inputs, outputs=dense_layer_2)

In [ ]:
model3.compile(loss='binary_crossentropy', optimizer='adam', metrics=[get_f1])
history = model3.fit(X_train2, y_train2[:,18:], epochs=5, verbose=1, validation_split=0.1)

In [ ]:
score = model3.evaluate(X_test2, y_test2[:,18:], verbose=1)

print("Test Score:", score[0])
print("Test F1 Accuracy:", score[1])

In [ ]:
result = model2.predict(X_test2)
for i in result:
    i[i>=0.5] = 1
    i[i<0.5] = 0
np.max(result)

In [ ]:
for i in range(0,0):
    print(i)

In [ ]:
y_test2[:18]

In [ ]:
pd.DataFrame(y_test2[:,18:])

### 3.3 Add more samples by mixing up the sentences in X_train and creating duplicates

In [ ]:
from PaddedClinicalBERTEmbeddings import PaddedClinicalBERTEmbeddings
embedding = PaddedClinicalBERTEmbeddings('negative for the following:  Fevers,\nchills, nausea, vomiting, night sweats, change in weight')